In [1]:
import pandas as pd
import optuna
import functools
import warnings
from tqdm import tqdm
warnings.simplefilter('ignore')


from sklearn.model_selection import train_test_split

#こいつがマジで有能
import optuna.integration.lightgbm as lgb

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

%matplotlib 


#dataの読み込み
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
submit_df = pd.read_csv("submit_sample.csv",header = None)


#訓練データ、テストデータがわかるようにダミーの目的変数を代入
test_df["y"] = -999

#訓練データ、テストデータを結合
all_df = pd.concat([train_df,test_df])
del train_df , test_df

all_df.reset_index(inplace=True)


#月末フラグ
month_list = list(all_df["month"])
day_list = list(all_df["day"])

end_of_month_flag = []
for i in range(len(all_df)):
    if day_list[i] in [30,31]:
        end_of_month_flag.append(1)
    elif day_list[i] == 29 and month_list[i] in ["feb","apr","jun","sep","nov"]:
        end_of_month_flag.append(1)
    elif day_list[i] == 28 and month_list[i] == "feb":
        end_of_month_flag.append(1)
    else:
        end_of_month_flag.append(0)

all_df["end_of_month_flag"] = end_of_month_flag


#月(month列)を数値に変換
month_dict = {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12}
month_int = [month_dict[all_df["month"][i]] for i in range(len(all_df))]
all_df["month"] = month_int

#month_day列作成
#決定木は大小関係さえわかればいいのでintのまま扱って問題ないと思っています
month_day = []
month_day = all_df["month"]*100 + all_df["day"]
all_df["month_day"] = month_day

#不要列削除
del all_df["index"]
del all_df["id"]
del all_df["pdays"]
del all_df["balance"]

#カテゴリカルのカラム名を指定
categorical_features = ["job","marital","education","default","housing","loan","contact","month","poutcome","end_of_month_flag"]


#ラベルエンコード
#target encodingもしましたがスコアは変わらなかったです

for col in categorical_features:
  lbl = preprocessing.LabelEncoder()
  lbl.fit(all_df[col])
  lbl.transform(all_df[col])
  all_df[col] = lbl.transform(all_df[col])


#訓練データ、テストデータの分割
train_df = all_df[all_df["y"] != -999]
test_df = all_df[all_df["y"] == -999]

#説明変数と目的変数に分ける
origin_y_train = train_df["y"]
origin_X_train = train_df.drop(["y"],axis = 1)
origin_X_test = test_df.drop(["y"],axis = 1)

output_df = pd.DataFrame()


Using matplotlib backend: TkAgg


In [2]:
##ここから先の実行は私のPC(第10世代corei5 メモリ8GB)では10数時間かかりました

for i in tqdm(range(100)):
    #訓練データからテストデータを分割
    X_train , X_valid , y_train , y_valid = train_test_split(origin_X_train,origin_y_train,test_size = 0.3 , random_state = i , stratify=origin_y_train)
    #データセット作成
    lgb_train = lgb.Dataset(X_train,y_train,categorical_feature = categorical_features,free_raw_data=False)
    lgb_eval = lgb.Dataset(X_valid , y_valid , reference = lgb_train , categorical_feature = categorical_features,free_raw_data=False)
    params ={"objective":"binary",
             "metric":"auc"    
    }

    best_params, tuning_history = dict(), list()
    booster = lgb.train(params, lgb_train, valid_sets=[lgb_train,lgb_eval],
                        verbose_eval=0,
                        best_params=best_params,
                        tuning_history=tuning_history)

    print("Best Params:", best_params)
    print("Tuning history:", tuning_history)

    #最もAUCの高いハイパーパラメーターを追加
    params.update(best_params)

    #学習
    model = lgb.train(
        params,lgb_train,
        valid_sets=[lgb_train,lgb_eval],
        verbose_eval = 10,
        num_boost_round = 1000,
        early_stopping_rounds=10
    )

    #コンペデータで予測
    y_pred = model.predict(origin_X_test,num_iteration=model.best_iteration)

    output_df[i] = y_pred


# submit_df = pd.read_csv("../../data/submit_sample.csv",header = None)
# del submit_df[1]
# submit_df["1"] = output_df.mean(axis = 1)
# submit_df
# submit_df.to_csv("../../output/result.csv",header=None,index=False)

# #試行錯誤中にfeature importanceを確認して特徴量エンジニアリングしていました
# lgb.plot_importance(model, height=0.5, figsize=(8,12))

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]


TypeError: __init__() got an unexpected keyword argument 'best_params'